In [ ]:
# Import need python packages
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import time, datetime

# Load environment variables from .env file
load_dotenv()

# Your Google Maps Places API key
API_KEY = os.getenv("API_KEY")

# Function to fetch data from the Places API
def fetch_places(api_key, query, url):
    # Initialize an empty list to hold the results
    all_results = []
    
    # Initial request parameters
    params = {
        "key": api_key,
        "query": query,
        "inputtype": "textquery"
    }
    
    while True:
        # Send the request and get the response
        response = requests.get(url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            
            # Append the results to the list
            all_results.extend(data.get('results', []))
            
            # Check if there's a next_page_token
            next_page_token = data.get('next_page_token')
            if next_page_token:
                # Update the parameters with the next_page_token
                params['pagetoken'] = next_page_token
                
                # Google recommends a short pause before requesting the next page
                time.sleep(2)
            else:
                break
        else:
            print(f"Error: {response.status_code} - {response.text}")
            break
    
    return all_results

# Function to fetch additional details for a place
def fetch_place_details(api_key, place_id, details_url):
    params = {
        "key": api_key,
        "place_id": place_id,
        "fields": "name,formatted_address,rating,user_ratings_total,place_id,formatted_phone_number,website"
    }
    
    # Send the request and get the response
    response = requests.get(details_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response and return the result
        return response.json().get('result', {})
    else:
        print(f"Error fetching details for {place_id}: {response.status_code} - {response.text}")
        return {}

# The query and URL for the Places API
query = os.getenv("QUERY")
url = os.getenv("TEXT_SEARCH_URL")
details_url = os.getenv("DETAILS_URL")

# Fetch all results from the initial search
all_results = fetch_places(API_KEY, query, url)

# Initialize a list to hold detailed results
detailed_results = []

# Fetch additional details for each place
for place in all_results:
    place_id = place.get('place_id')
    if place_id:
        details = fetch_place_details(API_KEY, place_id, details_url)
        detailed_results.append(details)

# Create a DataFrame from the detailed results
df = pd.DataFrame(detailed_results)

# Select and rename relevant columns
df = df[['name', 'formatted_address', 'rating', 'user_ratings_total', 'place_id', 'formatted_phone_number', 'website']]
df.columns = ['Name', 'Address', 'Rating', 'Total Ratings', 'Place ID', 'Phone Number', 'Website']

# Display the DataFrame
print(df)

# Get date for file name
current_date = datetime.date.today()

# Format the date as YYYYMMDD
formatted_date = current_date.strftime("%Y%m%d")

# Save to a CSV file
df.to_csv(formatted_date + ' ' + query + '.csv', index=False)

